In [1]:
%pip install pandas
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sleeperpy import User

In [3]:
matchups = pd.read_parquet('s3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper/matchups/')

In [4]:
owner_list = []
for owner_id in matchups['owner_id'].unique():
    owner_list.append(User.get_user(owner_id))
owners = pd.DataFrame(owner_list)
matchups = pd.merge(matchups, owners, left_on='owner_id', right_on='user_id', how='inner')

In [57]:
def calculate_points_against(row):
    points_against = matchups.loc[matchups['year'] == row['year']].loc[matchups['week'] == row['week']].loc[matchups['matchup_id'] == row['matchup_id']].loc[matchups['owner_id'] != row['owner_id']]['points'].values
    if len(points_against) == 1:
        return points_against[0]
    else:
        return 0
matchups['points_against'] = matchups.apply(calculate_points_against, axis=1)

In [75]:
def calculate_win_loss(row):
    points_against = matchups.loc[matchups['year'] == row['year']].loc[matchups['week'] == row['week']].loc[matchups['matchup_id'] == row['matchup_id']].loc[matchups['owner_id'] != row['owner_id']]['points'].values
    if len(points_against) == 1:
        if points_against < row['points']:
            return 1.0
        else:
            return 0.0
    else:
        return 0.0
matchups['actual_win_loss'] = matchups.apply(calculate_win_loss, axis=1)

In [78]:
# expected wins
def calculate_expected_wins(row):
    other_player_points = list(matchups.loc[matchups['year'] == row['year']].loc[matchups['week'] == row['week']].loc[matchups['username'] != row['username']]['points'])
    expected_wins = 0.0
    for other_player_point in other_player_points:
        if other_player_point < row['points']:
            expected_wins += 1
    return expected_wins
matchups['expected_wins'] = matchups.apply(calculate_expected_wins, axis=1)

In [79]:
matchups.loc[matchups['type'] == 'regular'].loc[matchups['year'] == 2023].loc[matchups['username'] == 'exoentropy']

,co_owners,keepers,league_id,owner_id,player_map,players_x,reserve,roster_id,starters_x,taxi,...,summoner_name,summoner_region,token,user_id,username,verification,points_against,win_loss,actual_win_loss,expected_wins
479,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,71.85,1.0,1.0,5.0
480,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,89.67,1.0,1.0,11.0
481,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,100.99,0.0,0.0,2.0
482,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,113.62,0.0,0.0,5.0
483,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,115.11,1.0,1.0,9.0
484,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,110.84,0.0,0.0,4.0
489,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,135.41,0.0,0.0,7.0
490,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,130.07,1.0,1.0,9.0
491,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,78.14,1.0,1.0,9.0
492,None,"[6794, 8151]",961113588070985728,586035702630752256,None,"[11199, 3294, 4217, 5846, 5927, 6794, 6826, 68...",None,7,"[3294, 8151, 9509, 5846, 6794, 4217, 7588, 592...",None,...,None,None,None,586035702630752256,exoentropy,None,145.94,0.0,0.0,0.0


In [82]:
matchups.loc[matchups['type'] == 'regular'].groupby(['username', 'year'])['actual_win_loss'].sum()

username       year
acarlough      2020     7.0
               2021     5.0
               2022    10.0
               2023     8.0
almostwinning  2020     6.0
               2021     3.0
               2022     6.0
               2023     3.0
bigoledoinks   2020    10.0
               2021     6.0
               2022     5.0
               2023     9.0
callentrail    2020     7.0
               2021     7.0
               2022     8.0
               2023     6.0
coolguytrav    2020    10.0
               2021     7.0
               2022     9.0
               2023     7.0
cramseyio      2020     5.0
               2021     7.0
               2022     6.0
               2023     6.0
exoentropy     2020     4.0
               2021    11.0
               2022     9.0
               2023     8.0
jyarrow        2020     5.0
               2021     8.0
               2022     5.0
               2023     5.0
lrich91        2020     4.0
               2021     6.0
               2022     6.0


In [83]:
matchups.loc[matchups['type'] == 'regular'].groupby(['username', 'year'])['expected_wins'].sum()

username       year
acarlough      2020     81.0
               2021     70.0
               2022     61.0
               2023     93.0
almostwinning  2020     66.0
               2021     44.0
               2022     84.0
               2023     59.0
bigoledoinks   2020     83.0
               2021     66.0
               2022     73.0
               2023     73.0
callentrail    2020     95.0
               2021     85.0
               2022     84.0
               2023     76.0
coolguytrav    2020    104.0
               2021     67.0
               2022     79.0
               2023     92.0
cramseyio      2020     62.0
               2021     76.0
               2022     76.0
               2023     70.0
exoentropy     2020     50.0
               2021    113.0
               2022    104.0
               2023     76.0
jyarrow        2020     48.0
               2021    100.0
               2022     59.0
               2023     77.0
lrich91        2020     52.0
               2021    